In [27]:
import pandas as pd
import random
import numpy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [28]:
#Creating random datasets
l_type=['Funny','Travel','Food','Fashion','News','Funny','Sports','Motivational','Music','Dance' ]
l_location=['Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chhattisgarh', 'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jharkhand', 'Karnataka', 'Kerala', 'Madhya Pradesh', 'Maharashtra',  'Manipur', 'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha', 'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana', 'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal']

types = [random.choice(l_type) for i in range(1000)]
location=[random.choice(l_location) for i in range(1000)]

data = {
    "Location": location,
    "types":types,
}
df = pd.DataFrame(data)
df.to_csv('/content/sample_data/ReelType.csv', mode='w')
# df

In [29]:
df = pd.read_csv('/content/sample_data/ReelType.csv')
df

,Unnamed: 0,Location,types
0,0,Haryana,Music
1,1,Mizoram,News
2,2,Arunachal Pradesh,Funny
3,3,Manipur,Music
4,4,Odisha,News
...,...,...,...
995,995,Madhya Pradesh,Dance
996,996,Nagaland,News
997,997,Jharkhand,Music
998,998,Rajasthan,News


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1000 non-null   int64 
 1   Location    1000 non-null   object
 2   types       1000 non-null   object
dtypes: int64(1), object(2)
memory usage: 23.6+ KB


In [31]:
df.groupby('types').size()

types
Dance            99
Fashion         107
Food            108
Funny           212
Motivational     95
Music            84
News            112
Sports           85
Travel           98
dtype: int64

In [32]:
df.groupby('Location').size()

Location
Andhra Pradesh       37
Arunachal Pradesh    35
Assam                40
Bihar                35
Chhattisgarh         27
Goa                  42
Gujarat              33
Haryana              38
Himachal Pradesh     33
Jharkhand            35
Karnataka            36
Kerala               39
Madhya Pradesh       31
Maharashtra          33
Manipur              39
Meghalaya            40
Mizoram              30
Nagaland             42
Odisha               30
Punjab               41
Rajasthan            34
Sikkim               36
Tamil Nadu           33
Telangana            35
Tripura              36
Uttar Pradesh        35
Uttarakhand          40
West Bengal          35
dtype: int64

In [33]:
df_new = df[df['types'] == 'Travel']
df_new

,Unnamed: 0,Location,types
19,19,Odisha,Travel
35,35,Kerala,Travel
64,64,Karnataka,Travel
80,80,Gujarat,Travel
86,86,Telangana,Travel
...,...,...,...
971,971,Kerala,Travel
981,981,Andhra Pradesh,Travel
988,988,Manipur,Travel
989,989,Arunachal Pradesh,Travel


In [34]:
df_new.groupby('Location').size()

Location
Andhra Pradesh       6
Arunachal Pradesh    5
Assam                2
Bihar                6
Chhattisgarh         3
Goa                  4
Gujarat              4
Haryana              4
Himachal Pradesh     1
Jharkhand            1
Karnataka            6
Kerala               9
Madhya Pradesh       3
Maharashtra          3
Manipur              3
Meghalaya            2
Nagaland             5
Odisha               6
Punjab               2
Rajasthan            2
Sikkim               3
Tamil Nadu           1
Telangana            3
Tripura              2
Uttar Pradesh        4
Uttarakhand          2
West Bengal          6
dtype: int64

In [35]:
df_new = df[df['types'] == 'Travel']
df_new
l_travel = df_new['Location'].tolist()
df_new.groupby('Location').size()
print(l_travel)

['Odisha', 'Kerala', 'Karnataka', 'Gujarat', 'Telangana', 'West Bengal', 'Uttar Pradesh', 'Odisha', 'Arunachal Pradesh', 'Andhra Pradesh', 'Arunachal Pradesh', 'West Bengal', 'Tripura', 'Punjab', 'Maharashtra', 'Gujarat', 'Odisha', 'Sikkim', 'Chhattisgarh', 'Goa', 'Assam', 'Madhya Pradesh', 'Bihar', 'Haryana', 'Kerala', 'Tamil Nadu', 'Sikkim', 'Manipur', 'Madhya Pradesh', 'Jharkhand', 'Odisha', 'Odisha', 'Bihar', 'Manipur', 'Madhya Pradesh', 'West Bengal', 'Andhra Pradesh', 'Andhra Pradesh', 'Gujarat', 'Karnataka', 'Himachal Pradesh', 'Odisha', 'Kerala', 'Maharashtra', 'Meghalaya', 'Arunachal Pradesh', 'Karnataka', 'Uttar Pradesh', 'Nagaland', 'Nagaland', 'Chhattisgarh', 'Nagaland', 'Meghalaya', 'West Bengal', 'Chhattisgarh', 'Uttarakhand', 'Uttar Pradesh', 'Punjab', 'Goa', 'Nagaland', 'Bihar', 'Bihar', 'Telangana', 'Kerala', 'Uttar Pradesh', 'Kerala', 'Haryana', 'Haryana', 'Gujarat', 'Uttarakhand', 'Goa', 'Rajasthan', 'Karnataka', 'Tripura', 'Telangana', 'Sikkim', 'Andhra Pradesh', 'B

In [41]:
# Load the data
tourist_data = pd.read_csv('/content/sample_data/TouristSpots.csv')
place = input('Enter the destination State:- ' )
# # Define the user preferences
# d = random.choice(l_travel)
# destination = d
# duration = '5 days'
# budget = 50000

# Filter the tourist data based on the destination and preferred activities
filtered_data = tourist_data[(tourist_data['States'] == place)]

# Sort the filtered data by rating and price
tourist_data = tourist_data.dropna(subset=['States'])#Drops NULL value
sorted_data = filtered_data.sort_values(by=['rating', 'price'], ascending=False)
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(sorted_data['States'])

# Define a function to calculate cosine similarity between rows
def get_similarities(place, data,tfidf):
    # Get the index of the input place
    place_matrix = tfidf.transform([place])
    # Get the cosine similarity scores for the input place
    sim_scores = list(cosine_similarity(place_matrix, data).flatten())
    # Return the cosine similarity scores
    return sim_scores


def recommend_place(place, data=sorted_data,tfidf=tfidf, n=10):
  if tourist_data[tourist_data['States']== place].shape[0] == 0:
    print('This place is either not so popular or you\have entered invalid_name.\n')

    for place in data.sample(n=n)['States'].values:
      print(place)
    return
  similarity_scores = get_similarities(place, tfidf_matrix,tfidf)
  data['similarity_factor']= similarity_scores
  
  print('\nHere is the list of the best places for you     to visit: \n')

  data.sort_values(by=['similarity_factor','rating'],ascending=[False, False],inplace=True)
  display(data[['places', 'price', 'rating','duration']].head(n))

  TotalCost = 0
  TotalDays = 0
  for p,row in data[['States', 'places', 'price', 'rating','duration']].head(n).iterrows():
    # print(f"{row['places']}")
    TotalCost = TotalCost + row['price']
    TotalDays = TotalDays + row['duration']
  print(f"\nThe tour will be of {int(TotalDays)} days and will cost around Rs.{int(TotalCost)}.")

  #Model Evaluation Function Call
  evaluateModel()

  #Updating Indivisual Rating of Sourist Spots
  # for p,row in data[['States', 'places', 'price', 'rating','duration']].head(n).iterrows():
  #   print(tourist_data.loc[row['place'], 'rating'])








# Check the efficiency of the recommendation Model
def evaluateModel():
  df_model = pd.read_csv('/content/sample_data/Model.csv')
  satisfied = input("Are you satisfied(y/n) : ")

  if satisfied == "y":
    df_model['positive'] = df_model['positive'].values + 1
    df_model['total'] = df_model['total'].values + 1
    df_model['rating'] = df_model['positive'].values / df_model['total'].values

  elif satisfied == "n":
    df_model['total'] = df_model['total'].values + 1
    df_model['rating'] = df_model['positive'].values / df_model['total'].values

  df_model.to_csv('/content/sample_data/Model.csv', mode='w')
  print(f"Current Efficiency of the system is : {df_model['rating'].values}")


recommend_place(place)
# evaluateModel()


Enter the destination State:- Odisha

Here is the list of the best places for you     to visit: 



,places,price,rating,duration
89,Chilika Lake,3000.0,10.0,3.0
87,Bhubaneswar,5000.0,8.0,1.0
90,Cuttack,5000.0,3.0,1.0
88,Konark,10000.0,2.0,3.0
86,Puri,4000.0,1.0,1.0



The tour will be of 9 days and will cost around Rs.27000.
Are you satisfied(y/n) : y
Current Efficiency of the system is : [0.83333333]


In [ ]:
#Updatind Dataset
for a in range(1, 4):
  tourist_data.loc['']

  for p,row in data[['States', 'places', 'price', 'rating','duration']].head(n).iterrows():
    # print(f"{row['places']}")
    TotalCost = TotalCost + row['price']
    TotalDays = TotalDays + row['duration']

In [ ]:
# data = {
#     "rating":[0],
#     "positive":[0],
#     "total":[0],
# }
# df = pd.DataFrame(data)
# df.to_csv('/content/sample_data/Model.csv', mode='w')